# Make 3D mixed shape training data with boundary interior methods¶
Loop into the directory Documents/unet/data/cellmodeller_simulation/to_use_GT/mix_label 
Inputs are 3D sub tif images for each class. 
Outputs are binary label image,
and corresponding five class: 1 being the background, 2/3 being the interior/boundary of the first class,
and 4/5 being the interior/boundary of the second class.

In [1]:
import os
import numpy as np
from skimage.io import imsave, imread
from skimage.segmentation import find_boundaries
import time, datetime
#save as .nii file
import nibabel as nib   
import pathlib

Here: specify the two classes you want to input. Class 1 is either bend or sphere: class 2 is always rod

In [2]:
class1 = 'sphere' # change to bend if needed
class2 =  'rod'
nameIDX = -(len(class1) + 6)

In [3]:
def get_boundaryInterior(image):
    image_size_x, image_size_y = image[1].shape
#   make a semantic mask
    semantic_masks = np.zeros(( len(image), image_size_x, image_size_y))   #, dtype = K.floatx())
#     print(semantic_masks.shape)
#     print(images.shape)
    
    edges = find_boundaries(image, mode = 'thick')
    interior = 2*(image > 0)
    semantic_mask = edges + interior
    semantic_mask[semantic_mask == 3] = 1
#   Swap category names - edges category 2, interior category 1, background category 0
    semantic_mask_temp = np.zeros(semantic_mask.shape, dtype = 'int')
    semantic_mask_temp[semantic_mask == 1] = 2
    semantic_mask_temp[semantic_mask == 2] = 1

    semantic_mask = semantic_mask_temp
    print(semantic_masks.shape)
    # save as nii
    binary_seg = np.transpose(semantic_mask)
    return binary_seg

change image directory as needed

In [4]:
mix_dir = os.path.join('data', 'mixShape')  
today = datetime.date.today()  
todaystr = today.isoformat()   
output_path= os.path.join('output', 'boundary_interior_output', todaystr) 
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True) # create directory if neccessary

In [5]:
for img in os.listdir(mix_dir):
    if img.find(class1) != -1:
        img_sphere = imread(os.path.join(mix_dir, img))
#         result = find(img,'_')
        name, ext = os.path.splitext(img)
        img_rod = imread(os.path.join(mix_dir, name[0:nameIDX] + 'rod_Label' + ext))
        print(img_sphere.shape)
        print(img_rod.shape)
        binarySeg_sphere = get_boundaryInterior(img_sphere)
        
        binarySeg_rod = get_boundaryInterior(img_rod)
        binarySeg_rod[binarySeg_rod == 1] = 3
        binarySeg_rod[binarySeg_rod == 2] = 4
        #combine
        binary_seg = binarySeg_rod + binarySeg_sphere
        
        # have problems sometimes output larger than '5' when overlap
        binary_seg[binary_seg > 4] = 4
        
        # make and save nifty images
        bseg = nib.Nifti1Image(binary_seg.astype(np.uint16), affine=np.eye(4))
         
        # make a new directory to save
        new_name = img.replace('_' + class1 + '_Label.tif','')   
        # change the saved file names
        nib.nifti1.save(bseg, os.path.join(output_path, new_name + '_Label.nii'))
        
    

(121, 221, 221)
(121, 221, 221)
(121, 221, 221)
(121, 221, 221)
(121, 221, 221)
(121, 221, 221)
(121, 221, 221)
(121, 221, 221)
